# Will Barker's Answers to the Shopify Data Science Internship Challenge

# Question 1

**On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one
model of shoe. We want to do some analysis of the average order value (AOV). When
we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13.
Given that we know these shops are selling sneakers, a relatively affordable item,
something seems wrong with our analysis.**

**A. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.**

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv")

In [3]:
data.head() # a quick look at first 5 rows... we can see order amounts much lower than $3000

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [4]:
data['order_amount'].mean() # naive AOV calculation

3145.128

The AOV being so much higher than expected (in combination with our prior knowledge about sneaker prices) indicates that there might be some outlier orders in the dataset. Since order amount is the product of the total items purchased and their price, these outliers could be the result of three things happening:

1. Some orders may have abnormally high-priced items
2. There may be orders with abnormally high volumes of total items on them
3. 1 & 2 happening at the same time

If we're only interested in an approximate AOV across all of the shops, a quick way to evaluate this would be to take the median order amount value. If outliers are present in the data the median won't be sensitive to them. Also, if the median is close to our naive AOV then it provides evidence that some of our underlying assumptions might be wrong (e.g. perhaps many more sneakers are sold than originally thought).

In [5]:
data['order_amount'].median() # median calculation

284.0

The median order amount is much lower and seems reasonable. Compared to the AOV calculated earlier, it suggests that there are indeed some outliers present in the data. However, the median on its own doesn't provide any insight into what the outliers are or why they're occurring. It also doesn't tell us anything about the performance of individual shops and their sneakers.

**B. What metric would you report for this dataset?**

If we're only interested in a single summary metric for this dataset, I think the median order amount is an good estimate of how our shops are doing. A better metric for average performance would be the AOV with all outliers removed from the data, since they aren't representive of our shops' normal and expected activity.

However, given that each shop only sells one kind of sneaker, I think it would be wise to calculate the average order value per shop. Comparing these averages would show which shops and their sneakers generate the most revenue, giving us an idea of what seems to be popular amongst customers. We can also look at each shop's sneaker price and average total orders to gain a better understanding about its performance. This could show where the outliers are; perhaps certain shops have much higher sales volumes or item prices.

Before we group the data by shops, we can quickly extract the item price for each sneaker by dividing each order's amount by its total items:

In [6]:
data['item_price'] = data['order_amount'] / data['total_items']
data.tail() # look at last 5 rows

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,item_price
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17,165.0
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16,117.0
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42,117.0
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18,177.0
4999,5000,44,734,288,2,debit,2017-03-18 15:48:18,144.0


We can now group the data to show shop averages for order amount and total items by creating a pivot table with **shop_id** as the index and setting the aggregate function for the values to be the mean. We can also use this function to display the item price for each shop's sneaker, since its average will just be equal to the true price:

In [10]:
shops = data.pivot_table(values=['order_amount', 'total_items', 'item_price'], index='shop_id', aggfunc='mean')

In [26]:
shops[:5] # showing the first 5 shops, in order of their shop IDs

,item_price,order_amount,total_items
shop_id,,,
1,158.0,308.818182,1.954545
2,94.0,174.327273,1.854545
3,148.0,305.250000,2.062500
4,128.0,258.509804,2.019608
5,142.0,290.311111,2.044444


Let's sort this aggregated data to see which shops have the highest average order amounts:

In [27]:
shops.sort_values(by='order_amount', ascending=False)[:5] # first 5 shops ranked by order_amount

,item_price,order_amount,total_items
shop_id,,,
42,352.0,235101.490196,667.901961
78,25725.0,49213.043478,1.913043
50,193.0,403.545455,2.090909
90,178.0,403.224490,2.265306
38,190.0,390.857143,2.057143


Sorted by average item price:

In [28]:
shops.sort_values(by='item_price', ascending=False)[:5] # first 5 shops ranked by item_price

,item_price,order_amount,total_items
shop_id,,,
78,25725.0,49213.043478,1.913043
42,352.0,235101.490196,667.901961
12,201.0,352.698113,1.754717
89,196.0,379.147541,1.934426
99,195.0,339.444444,1.740741


Sorted by average total items:

In [29]:
shops.sort_values(by='total_items', ascending=False)[:5] # first 5 shops ranked by total_items

,item_price,order_amount,total_items
shop_id,,,
42,352.0,235101.490196,667.901961
37,142.0,340.208333,2.395833
24,140.0,320.727273,2.290909
90,178.0,403.224490,2.265306
10,148.0,332.301887,2.245283


Immediately we see two clear outliers here with shops 42 and 78. Shop 42 has an average order amount of \$235,101.49 and an average of 668 items per order. It would be amazing to see one of our shops doing this well, if it wasn't so alarmingly suspicious. Shop 78's sneaker price of \$25,725 is the cause of its high order amount; this must be some sort of pricing error, unless they're selling resale Yeezys.

Let's look at shop 42's orders to see if we can figure out what's going on there:

In [17]:
data[data['shop_id'] == 42]

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,item_price
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,352.0
40,41,42,793,352,1,credit_card,2017-03-24 14:15:41,352.0
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00,352.0
308,309,42,770,352,1,credit_card,2017-03-11 18:14:39,352.0
409,410,42,904,704,2,credit_card,2017-03-04 14:32:58,352.0
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00,352.0
834,835,42,792,352,1,cash,2017-03-25 21:31:25,352.0
835,836,42,819,704,2,cash,2017-03-09 14:15:15,352.0
938,939,42,808,1056,3,credit_card,2017-03-13 23:43:45,352.0
979,980,42,744,352,1,debit,2017-03-12 13:09:04,352.0


Looking at the data, we can see that shop 42 has some suspicious activity that follows a clear pattern: user 607 made repeat purchases of 2000 sneakers through a credit card at 4 AM several times over our 30 day window. This looks like it could be a case of fraud and should be further investigated.

As for shop 78's high item price, we can do a quick check for unique values to see if $25,725 is the true item price or the average of several prices:

In [19]:
data[data['shop_id'] == 78]['item_price'].unique()

array([25725.])

No other prices exist, so it appears that the entirety of shop 78's transactions have the same pricing error on them. 

Looking back at our sorted pivot tables, it appears that shops 42 and 78 contained the only outliers in the dataset. With the outliers now known, we can remove them and recalculate the AOV.

**C. What is its value?**

To recalculate the AOV we'll use a cleaned dataset with all of shop 78's highly priced items and user 607's suspicious orders removed: 

In [44]:
data_shop78_removed = data[data['shop_id'] != 78] # filtering out shop 78s orders
data_cleaned = data_shop78_removed[data_shop78_removed['user_id'] != 607] # filering suspect orders from user 607

clean_pct = (round(len(data_cleaned)/len(data), 3))*100 # calculate the size of our cleaned dataset

print("Our cleaned dataset contains {}% of the original dataset.".format(clean_pct))

Our cleaned dataset contains 98.7% of the original dataset.


Now all that's left to do is take the new AOV:

In [39]:
data_cleaned['order_amount'].mean()

302.58051448247926

This AOV of \$302.58 seems much more reasonable, and is relatively close to our median order amount of \$284.

# Question 2

**A - How many orders were shipped by Speedy Express in total?**

SELECT Shippers.ShipperName, COUNT(Orders.OrderID)

FROM Orders JOIN Shippers

ON Orders.ShipperID = Shippers.ShipperID

WHERE Shippers.ShipperName = "Speedy Express"

GROUP BY Orders.ShipperID

**Answer:** 54

**B - What is the last name of the employee with the most orders?**

SELECT COUNT(OrderID), LastName

FROM Orders JOIN Employees

ON Orders.EmployeeID = Employees.EmployeeID

GROUP BY Orders.EmployeeID

ORDER BY ~COUNT(OrderID)

LIMIT 1

**Answer:** Peacock

**C - What product was ordered the most by customers in Germany?**

Note: I interpreted "ordered the most" to mean the product with the highest total quantity ordered.

SELECT ProductName, SUM(Quantity)

FROM Orders JOIN OrderDetails ON Orders.OrderID = OrderDetails.OrderID

JOIN Customers ON Orders.CustomerID = Customers.CustomerID

JOIN Products ON OrderDetails.ProductID = Products.ProductID

WHERE Country = 'Germany'

GROUP BY ProductName

ORDER BY ~SUM(Quantity)

LIMIT 1

**Answer:** Boston Crab Meat